In [ ]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import pickle
import numpy as np
import training
import config
import matplotlib.pyplot as plt
import FigureConfig as FC

In [ ]:
a = torch.load('./data/pReLU_power.ds')

In [ ]:
X, Y = a['X'], a['Y']
Xn, Yn = a['Xn'], a['Yn']
X_min, X_max = a['X_min'], a['X_max']
Y_min, Y_max = a['Y_min'], a['Y_max']

X_learn, Y_learn = a['X_learn'], a['Y_learn']
X_train, Y_train = a['X_train'], a['Y_train']
X_valid, Y_valid = a['X_valid'], a['Y_valid']
X_test , Y_test  = a['X_test'] , a['Y_test']

Xn_learn, Yn_learn = a['Xn_learn'], a['Yn_learn']
Xn_train, Yn_train = a['Xn_train'], a['Yn_train']
Xn_valid, Yn_valid = a['Xn_valid'], a['Yn_valid']
Xn_test , Yn_test  = a['Xn_test'] , a['Yn_test']

In [ ]:
train_data = TensorDataset(Xn_train, Yn_train)
valid_data  = TensorDataset(Xn_valid, Yn_valid)
test_data  = TensorDataset(Xn_test, Yn_test)

train_loader = DataLoader(train_data, batch_size=len(train_data))
valid_loader = DataLoader(valid_data, batch_size=len(valid_data))
test_loader = DataLoader(test_data, batch_size=len(test_data))

In [ ]:
SEEDs  = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
layers = [3, 4, 5, 6, 7, 8, 9, 10]
lrs    = [-3, -4, -5]

In [ ]:
lossfunction = torch.nn.MSELoss()

In [ ]:
results = torch.zeros([10, 8, 3, 3])

In [ ]:
for s, seed in enumerate(SEEDs):
    for l, layer in enumerate(layers):
        for m, lr in enumerate(lrs):
        
            exp_setup = f'{layer}_{lr}_{seed}'
            print(exp_setup)
            
            NN_temp = torch.load(f'./NNs/ReLUP_{exp_setup}.model', map_location='cpu')
            
            for x_train, y_train in train_loader:
                prediction_train = NN_temp(x_train)
            for x_valid, y_valid in valid_loader:
                prediction_valid = NN_temp(x_valid)
            for x_test, y_test in test_loader:
                prediction_test = NN_temp(x_test)
        
            loss_train = lossfunction(Yn_train, prediction_train)
            loss_valid = lossfunction(Yn_valid, prediction_valid)
            loss_test  = lossfunction(Yn_test,  prediction_test)
        
            results[s, l, m, 0] = loss_train
            results[s, l, m, 1] = loss_valid
            results[s, l, m, 2] = loss_test
        
            plt.figure(figsize=(12,12))
            plt.plot(np.linspace(0,1,100), np.linspace(0,1,100),  c='black')
            plt.scatter(Yn_train.flatten().numpy(), prediction_train.detach().flatten().numpy(), s=2, color=FC.Blue, label=f'train_loss: {loss_train:.2e}')
            plt.scatter(Yn_valid.flatten().numpy(), prediction_valid.detach().flatten().numpy(), s=2, color=FC.Green, label=f'valid_loss: {loss_valid:.2e}')
            plt.scatter(Yn_test.flatten().numpy(), prediction_test.detach().flatten().numpy(), s=2, color=FC.Pink, label=f'test_loss: {loss_test:.2e}')
            plt.xlim([0, 1])
            plt.ylim([0, 1])
            plt.legend(fontsize=12)
            plt.savefig(f'./NNs/final_ReLUP_{exp_setup}.pdf', format='pdf', bbox_inches='tight', transparent=True)

# Best model

In [ ]:
best_idx = torch.where(results == results[:,:,:,2].min())
print(results[:,:,:,2].min())
best_idx

# Save model for further use

In [ ]:
best_seed = SEEDs[best_idx[0]]
best_layer = layers[best_idx[1]]
best_lr = lrs[best_idx[2]]
best_seed, best_layer, best_lr

In [ ]:
exp_setup = f'{best_layer}_{best_lr}_{best_seed}'
exp_setup

In [ ]:
final_ReLUP = torch.load(f'./NNs/ReLUP_{exp_setup}.model', map_location='cpu')
print(lossfunction(Yn_test,  final_ReLUP(Xn_test)))

package = {'power_estimator': final_ReLUP,
           'X_min': X_min, 'X_max': X_max,
           'Y_min': Y_min, 'Y_max': Y_max}
torch.save(package, '../ReLU_power.package')